In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [2]:
import pandas as pd

# Corrected file names and variable names
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')


In [3]:
#Fake.head()
import pandas as pd

Fake = pd.read_csv("fake.csv")
Fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
import pandas as pd

true_news = pd.read_csv("True.csv")
print(true_news.head())


                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   


In [5]:
fake['class'] = 0   # Label all fake news as class 0
true['class'] = 1 

In [10]:
data=pd.concat([fake , true],axis=0)

In [11]:
data.sample(5)

,title,text,subject,date,class
11027,Former Trump campaign staffer files discrimina...,NEW YORK (Reuters) - A 26-year-old former camp...,politicsNews,"February 1, 2016",1
18962,NUCLEAR SHOWDOWN? NAVY SEAL TEAM That Killed O...,A WARNING shot has been fired at North Korea b...,left-news,"Mar 17, 2017",0
14153,UH OH! BILL CLINTON Unloads On Obama! [Video],"Unlike when I was president, a lot of things ...",politics,"Apr 7, 2016",0
19067,Turnout high as Iraqi Kurds defy threats to ho...,"ERBIL/SULAIMANIYA, Iraq (Reuters) - Kurds vote...",worldnews,"September 25, 2017",1
14743,Lebanon wants good ties with Saudi Arabia: for...,BEIRUT (Reuters) - Lebanese Foreign Minister G...,worldnews,"November 15, 2017",1


In [13]:
data=data.drop(["title","subject","date"],axis=1)

In [19]:
data.reset_index(inplace=True)

In [23]:
data.drop(['index'],axis=1,inplace=True)

In [25]:
data.sample(5)


,text,class
12690,Here s the evidence of just how much danger we...,0
4826,The Southern Poverty Law Center released a rep...,0
42605,BEIJING (Reuters) - The North Korean issue sho...,1
14187,Hillary Clinton told us she was dead broke b...,0
37534,GENEVA (Reuters) - The United Nations refugee ...,1


In [26]:
import re
import string

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\\W', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text


In [27]:
data["text"]=data["text"].apply(clean_text)

In [30]:
from sklearn.model_selection import train_test_split

# Independent and dependent variables
X = data["text"]
y = data["class"]

# Correct 4-way split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=24
)


In [34]:
vectorizer= TfidfVectorizer()
xv_train=vectorizer.fit_transform(X_train)
xv_test=vectorizer.transform(X_test)

In [36]:
lr=LogisticRegression()
lr.fit(xv_train,y_train)

LogisticRegression()

In [37]:
prediction = lr.predict(xv_test)

lr.score(xv_test, y_test)

0.9836971046770602

In [42]:
import joblib
import os

# Path to Desktop (macOS/Linux)
save_path = os.path.expanduser("~/Desktop/models")
os.makedirs(save_path, exist_ok=True)

# Save inside that folder
joblib.dump(vectorizer, os.path.join(save_path, "vectorizer1.jb"))
joblib.dump(lr, os.path.join(save_path, "lr_model1.jb"))

print("Models saved to:", save_path)


Models saved to: /Users/nakulmahajan/Desktop/models


In [44]:
import joblib
import numpy as np

vectorizer = joblib.load("models/vectorizer1.jb")
model = joblib.load("models/lr_model1.jb")

# Try a simple test
sample_texts = [
    "The government announced a new healthcare plan for citizens.",  # Real
    "Aliens have captured the White House and replaced the president.",  # Fake
]

X_test = vectorizer.transform(sample_texts)
pred = model.predict(X_test)
print(pred)


[0 0]
